In [11]:
import os
import sys
import time
import datetime
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import re
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from collections import Counter
sys.path.append('/Users/yantingting/PycharmProjects/modeling/')
import database_conncet as data_con
import data_processing as dp
import summary_statistics as ss
from data_io_utils import *

In [12]:
date = datetime.date.today().strftime('%m-%d')
file_path= os.path.join('/Users/yantingting/Seafile/风控/模型/10 印尼/低费率测试/ytt/', date)
if not os.path.exists(file_path):
    os.mkdir(file_path)

# 渠道代码

In [13]:
query4 = '''
SELECT
        case when c.product_code = 'UKU' then c.media_source else 'cashcash' end as media_source,
        c.channel,
        date(c.create_time) as reg_date,
		c.level_group as level_group,
		c.device,
		c.product_code,
		c.campaign,
        date(l.apply_time) as apply_date,
        l.device_approval,
        l.product_id,
		count(distinct c.id) as customer_install ,
		count(distinct case when l.return_flag = 'false'  then l.id end) as application ,
		count(distinct case when l.return_flag = 'false' and effective_date >= '2020-01-07' and loan_status not in ('DENIED', 'RESCIND', 'APPROVING', 'CREATED') then l.id else null end)as fangkuan
	from dw_gocash_go_cash_loan_gocash_core_customer c
	left join dw_gocash_go_cash_loan_gocash_core_loan l on c.id = l.customer_id
	where date(c.create_time)>='2020-01-18'
	group by 1,2,3,4,5,6,7,8,9,10;
'''

In [14]:
df4 = data_con.database_gn(query4)
save_data_to_pickle(df4, file_path, '渠道转化.pkl')
df4.shape

Successful connection


(44181, 13)

# 渠道分析

In [5]:
df4 = load_data_from_pickle( file_path, '渠道转化.pkl')
# df4['media_source'] = df4.apply(lambda x: x['channel'] if x['media_source']=='' else 'cashcash_int' if x['channel'] == 'cashcash_int' else x['media_source'],axis = 1)
df4['media_source'] = df4.apply(lambda x: x['media_source'] if x['media_source']=='' else 'cashcash_int' if x['channel'] == 'cashcash_int' else x['media_source'],axis = 1)
cols_meiti = ['Facebook Ads','GooglePlay','googleadwords_int','organic']
cols_bd = ['kreditpedia','cashcash_int','affle_int','xyads_int','dokteruang_int','estrend','unions','intellectads_int',
           'superads_int','tyroo_int','pokktperformance_int','Kredit All','invite_friend_activity_01','inmobi_int']
df4['source'] = df4['media_source'].apply(lambda x:x if x in cols_meiti else 'BD' if x in cols_bd else 'else')
df4['source_detail'] = df4['media_source'].apply(lambda x:x if x in cols_meiti else x if x in cols_bd else 'else')
cols2 = ['uku-fb-sm-MIN4-0115-cs',  'UKU-FB-sm-MIN2-M-0113',  'UKU-FB-sm-MIN2-V-0113',  'UKU-FB-sm-dl-02-fk-M-0114',
         'UKU Install 02','UKU-FB-sm-MIN2-M-1127','uku-tt-sm-MIN1-0626-2','UKU-MIN2-0424-1', 'UKU-MIN2-az-fk02',
         'UKU-MIN2-0423-1']
df4['ads'] = df4['campaign'].apply(lambda x:x if x in cols2 else 'else')
df4['source_flag'] = df4.apply(lambda x: '非调整渠道' if ((x['ads'] == 'else')&(x['source']!='BD')) else '调整渠道',axis = 1)
df4.shape

(44480, 17)

In [6]:
df_temp = df4[(df4['level_group']!='OneTest1')&(df4['level_group']!='OneTest2')]
df_temp['source'] = df_temp['source'].apply(lambda x:'BD' if x =='BD' 
                                            else '媒体' if (x == 'Facebook Ads') |(x == 'googleadwords_int')
                                            else  '自然')
frame1 = pd.pivot_table(df_temp,index = ('reg_date'),columns='source',values=('customer_install','fangkuan'),
               aggfunc='sum',fill_value=0)
frame1
frame2 = pd.pivot_table(df_temp[df_temp['source']=='媒体'],index = ('reg_date'),columns='source_flag',values=('customer_install','fangkuan'),
               aggfunc='sum',fill_value=0)
frame2
frame3 = pd.pivot_table(df_temp,index = ('reg_date'),columns='source_flag',values=('customer_install','fangkuan'),
               aggfunc='sum',fill_value=0)
frame3
# frame1.to_excel(os.path.join(file_path,'temp1.xlsx'))
# frame2.to_excel(os.path.join(file_path,'temp2.xlsx'))
frame3.to_excel(os.path.join(file_path,'temp3.xlsx'))

/Users/yantingting/anaconda3/envs/modeling/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


customer_install             fangkuan          
source                   BD    媒体    自然       BD   媒体   自然
reg_date                                                  
2020-01-18              781  5333  4158       69  496  232
2020-01-19              797  4577  3797       85  430  216
2020-01-20              960  5287  4257       81  531  285
2020-01-21             1004  4832  4363      102  433  242
2020-01-22             1211  5784  4609      124  655  256
2020-01-23             1321  5549  4520      132  543  212
2020-01-24             1408  4665  4323      115  374  197
2020-01-25             1413  4542  3836      122  366  186
2020-01-26             1062  4032  3657       88  350  138
2020-01-27             1181  4101  4194      106  367  246
2020-01-28             1198  4160  4221      100  339  214
2020-01-29             1143  4313  4250       95  302  237
2020-01-30             1098  4244  4091       88  332  191
2020-01-31             1263  4149  3797       86  354  199
2020-02-01             1138  4244  3707      108  374  181
2020-02-02              982  3731  3022       54  252  132
2020-02-03             1175  4364  4621       75  354  194
2020-02-04             1388  4385  4801       89  369  164
2020-02-05             1416  4222  4355       93  314  127
2020-02-06             1635  4444  4448       99  337   91
2020-02-07              637  4409  4809       38  327  143
2020-02-08              221  4155  4531       13  248  139
2020-02-09              371  3793  4794       14  197  103
2020-02-10                3    35    56        0    0    0

customer_install       fangkuan      
source_flag             调整渠道 非调整渠道     调整渠道 非调整渠道
reg_date                                         
2020-01-18               693  4640       55   441
2020-01-19               571  4006       31   399
2020-01-20               676  4611       35   496
2020-01-21               689  4143       41   392
2020-01-22               937  4847       75   580
2020-01-23               861  4688       49   494
2020-01-24               677  3988       37   337
2020-01-25               714  3828       33   333
2020-01-26               552  3480       28   322
2020-01-27               554  3547       21   346
2020-01-28               400  3760       17   322
2020-01-29               622  3691       27   275
2020-01-30               801  3443       42   290
2020-01-31               699  3450       29   325
2020-02-01               555  3689       34   340
2020-02-02               311  3420       12   240
2020-02-03               994  3370       36   318
2020-02-04               531  3854       28   341
2020-02-05               530  3692       18   296
2020-02-06               542  3902       22   315
2020-02-07               541  3868       10   317
2020-02-08               504  3651       23   225
2020-02-09               379  3414       14   183
2020-02-10                 7    28        0     0

customer_install       fangkuan      
source_flag             调整渠道 非调整渠道     调整渠道 非调整渠道
reg_date                                         
2020-01-18              1474  8798      124   673
2020-01-19              1368  7803      116   615
2020-01-20              1636  8868      116   781
2020-01-21              1693  8506      143   634
2020-01-22              2148  9456      199   836
2020-01-23              2182  9208      181   706
2020-01-24              2085  8311      152   534
2020-01-25              2127  7664      155   519
2020-01-26              1614  7137      116   460
2020-01-27              1735  7741      127   592
2020-01-28              1598  7981      117   536
2020-01-29              1765  7941      122   512
2020-01-30              1899  7534      130   481
2020-01-31              1962  7247      115   524
2020-02-01              1693  7396      142   521
2020-02-02              1293  6442       66   372
2020-02-03              2169  7991      111   512
2020-02-04              1919  8655      117   505
2020-02-05              1946  8047      111   423
2020-02-06              2177  8350      121   406
2020-02-07              1178  8677       48   460
2020-02-08               725  8182       36   364
2020-02-09               750  8208       28   286
2020-02-10                10    84        0     0

In [6]:
df4_temp = df4[df4['reg_date']>=pd.to_datetime('2020-01-23')]
df4_temp.head()
df4_temp.to_excel(os.path.join(file_path, 'temp2.xlsx'))

,media_source,channel,reg_date,level_group,device,product_code,campaign,apply_date,device_approval,product_id,customer_install,application,fangkuan,source,source_detail,ads,source_flag
9,appstore,appstore,2020-01-23,,IOS,UKU,,None,None,NaN,5,0,0,else,else,else,非调整渠道
10,appstore,appstore,2020-01-23,Test2,IOS,UKU,,None,None,NaN,1,0,0,else,else,else,非调整渠道
11,appstore,appstore,2020-01-24,,IOS,UKU,,None,None,NaN,3,0,0,else,else,else,非调整渠道
12,appstore,appstore,2020-01-24,Test2,IOS,UKU,,None,None,NaN,1,0,0,else,else,else,非调整渠道
13,appstore,appstore,2020-01-25,,IOS,UKU,,None,None,NaN,4,0,0,else,else,else,非调整渠道


In [7]:
df4_temp = df4_temp[(df4_temp['level_group'] != 'OneControlGroup')&(df4_temp['level_group'] != 'OneTest1')
            &(df4_temp['level_group'] != 'OneTest2')&(df4_temp['level_group'] != 'ControlGroup')]
frame1 = pd.pivot_table(df4_temp,index = ('source','source_detail'), 
               values=('customer_install','application','fangkuan'),aggfunc='sum',
              fill_value=0)
frame1['注册到申请'] = frame1['application']/frame1['customer_install']
frame1['申请到放款'] = frame1['fangkuan']/frame1['application']
frame1['注册到放款'] = frame1['fangkuan']/frame1['customer_install']
frame1 = frame1.fillna(np.nan)
frame1
frame1.to_excel(os.path.join(file_path,'temp.xlsx'))

application  customer_install  \
source            source_detail                                              
BD                affle_int                           25                57   
                  cashcash_int                       389               662   
                  dokteruang_int                     127               199   
                  estrend                             50                93   
                  inmobi_int                         951              1712   
                  invite_friend_activity_01          111               196   
                  kreditpedia                       2840              5371   
                  pokktperformance_int               149               240   
                  superads_int                         0                 4   
                  tyroo_int                            0                 1   
                  unions                             809              1363   
                  xyads_int                         3365              5746   
Facebook Ads      Facebook Ads                      4941              9368   
GooglePlay        GooglePlay                        2842             16119   
else              else                              9993             29278   
googleadwords_int googleadwords_int                26321             46528   
organic           organic                           8676             17723   

                                             fangkuan     注册到申请     申请到放款  \
source            source_detail                                             
BD                affle_int                         6  0.438596  0.240000   
                  cashcash_int                     80  0.587613  0.205656   
                  dokteruang_int                   18  0.638191  0.141732   
                  estrend                           3  0.537634  0.060000   
                  inmobi_int                      128  0.555491  0.134595   
                  invite_friend_activity_01        20  0.566327  0.180180   
                  kreditpedia                     376  0.528766  0.132394   
                  pokktperformance_int             20  0.620833  0.134228   
                  superads_int                      0  0.000000       NaN   
                  tyroo_int                         0  0.000000       NaN   
                  unions                          119  0.593544  0.147095   
                  xyads_int                       417  0.585625  0.123923   
Facebook Ads      Facebook Ads                    668  0.527434  0.135195   
GooglePlay        GooglePlay                      287  0.176314  0.100985   
else              else                           1244  0.341314  0.124487   
googleadwords_int googleadwords_int              3728  0.565702  0.141636   
organic           organic                        1245  0.489533  0.143499   

                                                注册到放款  
source            source_detail                        
BD                affle_int                  0.105263  
                  cashcash_int               0.120846  
                  dokteruang_int             0.090452  
                  estrend                    0.032258  
                  inmobi_int                 0.074766  
                  invite_friend_activity_01  0.102041  
                  kreditpedia                0.070006  
                  pokktperformance_int       0.083333  
                  superads_int               0.000000  
                  tyroo_int                  0.000000  
                  unions                     0.087307  
                  xyads_int                  0.072572  
Facebook Ads      Facebook Ads               0.071307  
GooglePlay        GooglePlay                 0.017805  
else              else                       0.042489  
googleadwords_int googleadwords_int          0.080124  
organic           organic                    0.070248

# 逾期率监测code

In [8]:
query2 = '''
select distinct
    loan_id,
	return_flag,
	approved_period,
	case
		when media_source in('kreditpedia',
		'cashcash_int',
		'affle_int',
		'xyads_int',
		'dokteruang_int',
		'unions',
		'intellectads_int',
		'superads_int',
		'tyroo_int',
		'pokktperformance_int',
		'Kredit All',
		'invite_friend_activity_01',
		'inmobi_int',
		'estrend') then media_source
		when media_source = 'Facebook Ads' then 'Facebook Ads'
		when media_source in('GooglePlay',
		'googleplay') then 'GooglePlay'
		when media_source = 'googleadwords_int' then 'googleadwords_int'
		when media_source = 'organic' then 'organic'
		else 'else'
end as channelgroup,
	groupup,
	case when late_date-effective_date = approved_period then 1 else 0 end as dpd1_flag
from
	(
	select
		t1.id loan_id,
		effective_date,
		return_flag,
		loan_status,
		approved_period,
		late_date,
		approved_principal,
		case
			when media_source is null
			or media_source = '' then channel
			else media_source
	end as media_source,
		case
			when "grouping" like '%ControlGroup%' and newusermodelscorev5::numeric<618 then 'ControlGroup_less618'
            when "grouping" like '%ControlGroup%' and newusermodelscorev5::numeric>=618 then 'ControlGroup_more618'
			when "grouping" like '%Test1%' then 'Test1'
			when "grouping" like '%Test2%' then 'Test2'
			when "grouping" like '%Test3%' then 'Test3'
			when "grouping" like '%Test4%' then 'Test4'
			else "grouping"
	end as groupup,
		case
			when loan_status = 'COLLECTION' then current_date-late_date
			else round(late_fee /(approved_principal*0.025))::int
	end as DPD
	from
		(
		select
			*
		from
			dw_gocash_go_cash_loan_gocash_core_loan
		where
			effective_date is not null
			and effective_date_accurate  >='2020-01-17 12:00:00'::timestamp
			and effective_date <= current_date-15
            and product_id='1'
            and approved_period<>'22'
            and device_approval='ANDROID'
			and loan_status not in ('DENIED',
			'RESCIND',
			'APPROVING',
			'CREATED')) t1
	left join (
		select
			*
		from
			(
			select
				order_id,
				late_date,
				row_number() over(partition by order_id
			order by
				late_date asc) as num
			from
				dw_gocash_gocash_collection_col_case
			where
				(order_status = 'COLLECTION_PAIDOFF'
				or order_status = 'COLLECTION')
				and app_id not in ('Credits',
				'KASANDAAI'))t
		where
			num = 1) t2 on
		t1.id = t2.order_id
	left join (
		select
			id,
			media_source
		from
			public.dw_gocash_go_cash_loan_gocash_core_customer ) t3 on
		t1.customer_id = t3.id
    left join (
		select
			loanid,
			newusermodelscorev5
		from
			rt_risk_mongo_gocash_installmentriskcontrolresult ) t4 on
		t1.id::varchar = t4.loanid
    )t 
    '''

In [10]:
df2 = data_con.database_gn(query2)
save_data_to_pickle(df2, file_path, '逾期率表现.pkl')
df2.shape

Successful connection


(15594, 6)

# 逾期率测试分析

In [11]:
df2_temp = df2[(df2['groupup'] != 'OneControlGroup')&(df2['groupup'] != 'ControlGroup_more618')
               &(df2['groupup'] != 'ControlGroup_less618')&(df2['groupup'] != 'ControlGroup')]
frame2 = pd.pivot_table(df2_temp,index = 'channelgroup', columns=('return_flag','dpd1_flag'),values='loan_id',aggfunc='count')
frame2['新客首逾'] = frame2[('false',1)]/(frame2[('false',1)] +frame2[('false',0)] )
frame2['老客首逾'] = frame2[('true',1)]/(frame2[('true',1)] +frame2[('true',0)] )
frame2

return_flag                 false            true             新客首逾      老客首逾
dpd1_flag                       0       1       0      1                    
channelgroup                                                                
Facebook Ads                330.0   241.0   333.0  119.0  0.422067  0.263274
GooglePlay                  378.0   250.0   695.0  321.0  0.398089  0.315945
Kredit All                    1.0     NaN     1.0    NaN       NaN       NaN
affle_int                    35.0    28.0    50.0   24.0  0.444444  0.324324
cashcash_int                 43.0    42.0    70.0   33.0  0.494118  0.320388
dokteruang_int               13.0    18.0    30.0   17.0  0.580645  0.361702
else                        103.0    64.0   160.0   60.0  0.383234  0.272727
estrend                       6.0     1.0     6.0    NaN  0.142857       NaN
googleadwords_int          1692.0  1067.0  1631.0  621.0  0.386734  0.275755
inmobi_int                   47.0    18.0    15.0    4.0  0.276923  0.210526
intellectads_int              NaN     1.0     1.0    NaN       NaN       NaN
invite_friend_activity_01     5.0     6.0     9.0    2.0  0.545455  0.181818
kreditpedia                 124.0   113.0   159.0   53.0  0.476793  0.250000
organic                     454.0   336.0   589.0  226.0  0.425316  0.277301
pokktperformance_int          8.0     6.0     5.0    3.0  0.428571  0.375000
superads_int                  5.0     1.0     8.0    1.0  0.166667  0.111111
tyroo_int                     4.0     3.0    12.0    9.0  0.428571  0.428571
unions                       20.0     7.0    13.0    3.0  0.259259  0.187500
xyads_int                    61.0    41.0    32.0   14.0  0.401961  0.304348

In [12]:
frame_temp = pd.merge(frame1, frame2, left_on = frame1.index.get_level_values(1),right_on = frame2.index, how = 'left')
frame_temp.to_excel(os.path.join(file_path, '转化和逾期.xlsx'))
frame_temp

/Users/yantingting/anaconda3/envs/modeling/lib/python3.7/site-packages/pandas/core/reshape/merge.py:617: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


,key_0,application,customer_install,fangkuan,注册到申请,申请到放款,注册到放款,"(false, 0)","(false, 1)","(true, 0)","(true, 1)","(新客首逾, )","(老客首逾, )"
0,affle_int,25,57,6,0.438596,0.240000,0.105263,35.0,28.0,50.0,24.0,0.444444,0.324324
1,cashcash_int,389,662,80,0.587613,0.205656,0.120846,43.0,42.0,70.0,33.0,0.494118,0.320388
2,dokteruang_int,127,199,18,0.638191,0.141732,0.090452,13.0,18.0,30.0,17.0,0.580645,0.361702
3,estrend,50,93,3,0.537634,0.060000,0.032258,6.0,1.0,6.0,NaN,0.142857,NaN
4,inmobi_int,951,1712,128,0.555491,0.134595,0.074766,47.0,18.0,15.0,4.0,0.276923,0.210526
5,invite_friend_activity_01,111,196,20,0.566327,0.180180,0.102041,5.0,6.0,9.0,2.0,0.545455,0.181818
6,kreditpedia,2840,5371,376,0.528766,0.132394,0.070006,124.0,113.0,159.0,53.0,0.476793,0.250000
7,pokktperformance_int,149,240,20,0.620833,0.134228,0.083333,8.0,6.0,5.0,3.0,0.428571,0.375000
8,superads_int,0,4,0,0.000000,NaN,0.000000,5.0,1.0,8.0,1.0,0.166667,0.111111
9,tyroo_int,0,1,0,0.000000,NaN,0.000000,4.0,3.0,12.0,9.0,0.428571,0.428571


# 利率和渠道的关系

In [ ]:
调整渠道 vs 非调整渠道

In [12]:
df4_1 = df4[(df4['level_group'] != 'OneControlGroup')&(df4['level_group'] != 'OneTest1')
            &(df4['level_group'] != 'OneTest2')&(df4['level_group'] != '对照组')]
frame1 = pd.pivot_table(df4_1,index = ('product_code', 'device','source','source_detail', 'level_group'), columns = 'source_flag',
               values=('customer_install','application','fangkuan'),aggfunc='sum',
              fill_value=0)
frame1['调整渠道占比'] = frame1[('application', '调整渠道')]/frame1['application'].apply(lambda x:x.sum(),axis = 1)
frame1[('注册到申请','调整渠道')] = frame1[('application', '调整渠道')]/frame1['customer_install', '调整渠道']
frame1[('注册到申请','非调整渠道')] = frame1[('application', '非调整渠道')]/frame1['customer_install', '非调整渠道']
frame1[('申请到放款','调整渠道')] = frame1[('fangkuan', '调整渠道')]/frame1[('application', '调整渠道')]
frame1[('申请到放款','非调整渠道')] = frame1[('fangkuan', '非调整渠道')]/frame1[('application', '非调整渠道')]
frame1[('注册到放款','调整渠道')] = frame1[('fangkuan', '调整渠道')]/frame1[('customer_install', '调整渠道')]
frame1[('注册到放款','非调整渠道')] = frame1[('fangkuan', '非调整渠道')]/frame1[('customer_install', '非调整渠道')]
frame1 = frame1.fillna(np.nan)
frame1
t1, p1 = stats.ttest_ind(frame1[('注册到申请','调整渠道')][~frame1[('注册到申请','调整渠道')].isna()],
                         frame1[('注册到申请','非调整渠道')][~frame1[('注册到申请','非调整渠道')].isna()])
print("注册到申请 t = " + str(t1))
print("注册到申请 p = " + str(p1))

t2, p2 = stats.ttest_ind(frame1[('申请到放款','调整渠道')][~frame1[('申请到放款','调整渠道')].isna()],
                         frame1[('申请到放款','非调整渠道')][~frame1[('申请到放款','非调整渠道')].isna()])
print("申请到放款 t = " + str(t2))
print("申请到放款 p = " + str(p2))

t3, p3 = stats.ttest_ind(frame1[('注册到放款','调整渠道')][~frame1[('注册到放款','调整渠道')].isna()],
                         frame1[('注册到放款','非调整渠道')][~frame1[('注册到放款','非调整渠道')].isna()])
print("注册到放款 t = " + str(t3))
print("注册到放款 p = " + str(p3))

frame1.to_excel(os.path.join(file_path,'temp1.xlsx'))


application  \
source_flag                                                                          调整渠道   
product_code device  source            source_detail             level_group                
UKU          ANDROID BD                affle_int                 ControlGroup          19   
                                                                 Test1                 20   
                                                                 Test2                 29   
                                                                 Test3                 21   
                                                                 Test4                 14   
                                       cashcash_int              ControlGroup          29   
                                                                 Test1                 68   
                                                                 Test2                 94   
                                                                 Test3                110   
                                                                 Test4                 97   
                                       dokteruang_int            ControlGroup          13   
                                                                 Test1                 30   
                                                                 Test2                 40   
                                                                 Test3                 31   
                                                                 Test4                 40   
                                       estrend                   ControlGroup           8   
                                                                 Test1                  9   
                                                                 Test2                  9   
                                                                 Test3                 16   
                                                                 Test4                 20   
                                       inmobi_int                                       9   
                                                                 ControlGroup         160   
                                                                 Test1                283   
                                                                 Test2                301   
                                                                 Test3                291   
                                                                 Test4                340   
                                       invite_friend_activity_01 ControlGroup          14   
                                                                 Test1                 46   
                                                                 Test2                 44   
                                                                 Test3                 30   
                                                                 Test4                 55   
                                       kreditpedia                                      0   
                                                                 ControlGroup         117   
                                                                 Test1                513   
                                                                 Test2                634   
                                                                 Test3                618   
                                                                 Test4                732   
                                       pokktperformance_int                             0   
                                                                 ControlGroup          23   
                                                                 Test1                 46   
                                                                 Test2         

注册到申请 t = 2.447210817895238
注册到申请 p = 0.01525974854124751
申请到放款 t = -1.4806992044113034
申请到放款 p = 0.14036069672963225
注册到放款 t = 0.09436173317801039
注册到放款 p = 0.9249162812380274


In [ ]:
Control vs. Test1

In [16]:
df6 = df4[(df4['level_group'] == 'ControlGroup')|(df4['level_group'] == 'Test1')]
frame6 = pd.pivot_table(df6, index = ['product_code', 'device','source', 'source_flag'], columns = 'level_group',
                        values= ['customer_install','application', 'fangkuan'],aggfunc='sum',fill_value=0)
frame6[('注册到申请','ControlGroup')] = frame6[('application', 'ControlGroup')]/frame6['customer_install', 'ControlGroup']
frame6[('注册到申请','Test1')] = frame6[('application', 'Test1')]/frame6['customer_install', 'Test1']
frame6[('申请到放款','ControlGroup')] = frame6[('fangkuan', 'ControlGroup')]/frame6[('application', 'ControlGroup')]
frame6[('申请到放款','Test1')] = frame6[('fangkuan', 'Test1')]/frame6[('application', 'Test1')]
frame6 = frame6.fillna(np.nan)
frame6
t1, p1 = stats.ttest_ind(frame6[('注册到申请','ControlGroup')][~frame6[('注册到申请','ControlGroup')].isna()],
                         frame6[('注册到申请','Test1')][~frame6[('注册到申请','Test1')].isna()])
                                          
print("注册到申请 t = " + str(t1))
print("注册到申请 p = " + str(p1))

t2, p2 = stats.ttest_ind(frame6[('申请到放款','ControlGroup')][~frame6[('申请到放款','ControlGroup')].isna()],
                         frame6[('申请到放款','Test1')][~frame6[('申请到放款','Test1')].isna()])
print("申请到放款 t = " + str(t2))
print("申请到放款 p = " + str(p2))
frame6.to_excel(os.path.join(file_path, 'temp3.xlsx'))

application        \
level_group                                        ControlGroup Test1   
product_code device  source            source_flag                      
UKU          ANDROID BD                调整渠道                 542  1835   
                     Facebook Ads      调整渠道                  71   150   
                                       非调整渠道                425   946   
                     GooglePlay        非调整渠道                  4   548   
                     else              非调整渠道                152   494   
                     googleadwords_int 调整渠道                 406   968   
                                       非调整渠道               2261  5430   
                     organic           非调整渠道                755  1738   
             IOS     else              非调整渠道                  2    11   
                     googleadwords_int 非调整渠道                  0     0   
                     organic           非调整渠道                114   365   
cashcash     ANDROID BD                调整渠道                  28   117   
                     Facebook Ads      调整渠道                   4    22   
                                       非调整渠道                 24    81   
                     else              非调整渠道                 16    91   
                     googleadwords_int 调整渠道                  13    38   
                                       非调整渠道                 86   217   
                     organic           非调整渠道                  6    25   

                                                   customer_install        \
level_group                                            ControlGroup Test1   
product_code device  source            source_flag                          
UKU          ANDROID BD                调整渠道                     967  3273   
                     Facebook Ads      调整渠道                      89   223   
                                       非调整渠道                    763  1963   
                     GooglePlay        非调整渠道                      8  1283   
                     else              非调整渠道                    263  1001   
                     googleadwords_int 调整渠道                     680  1734   
                                       非调整渠道                   3736  9720   
                     organic           非调整渠道                   1355  3748   
             IOS     else              非调整渠道                      3    27   
                     googleadwords_int 非调整渠道                      0     1   
                     organic           非调整渠道                    217   590   
cashcash     ANDROID BD                调整渠道                      34   151   
                     Facebook Ads      调整渠道                       4    22   
                                       非调整渠道                     36    96   
                     else              非调整渠道                     20   137   
                     googleadwords_int 调整渠道                      17    58   
                                       非调整渠道                    104   268   
                     organic           非调整渠道                      8    32   

                                                       fangkuan        \
level_group                                        ControlGroup Test1   
product_code device  source            source_flag                      
UKU          ANDROID BD                调整渠道                 132   247   
                     Facebook Ads      调整渠道                  24    39   
                                       非调整渠道                 94   137   
                     GooglePlay        非调整渠道                  0    63   
                     else              非调整渠道                 41    87   
                     googleadwords_int 调整渠道                  58    49   
                                       非调整渠道                640   916   
                     organic           非调整渠道                142   220   
             IOS     else              非调整渠道                  1     2   
                     goo

注册到申请 t = 1.3224678085163992
注册到申请 p = 0.1951051482712982
申请到放款 t = 2.28524148916323
申请到放款 p = 0.029069419423207007
